# Models

### import modules

In [ ]:
# Data, Plot and Statistics
import os
import shutil
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MaxNLocator
import seaborn as sns
import statsmodels.api as sm
import six
from sklearn.model_selection import train_test_split
# Descripotr Transformation
from rdkit import Chem
from rdkit.Chem import AllChem
import codecs
from SmilesPE.tokenizer import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
# Machine Leaning
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost
import shap

In [ ]:
file = pd.read_excel('./database/opv_ai_database_used.xlsx')

In [ ]:
file.head()

In [ ]:
file.info() # check data in each column is in the right form

In [ ]:
i = file.shape[0]
index = np.array(file['Index']).reshape(i,)
### Donor material
donor_name = np.array(file['Donor']).reshape(i,)
donor_smiles = np.array(file['Donor SMILES']).reshape(i,)
donor_homo = np.array(file['HOMO of Donor (eV)']).reshape(i,)
donor_lumo = np.array(file['LUMO of Donor (eV)']).reshape(i,)
donor_bandgap = np.array(file['Bandgap of Donor (eV)']).reshape(i,)
### Acceptor material
acceptor_name = np.array(file['Acceptor']).reshape(i,)
acceptor_smiles = np.array(file['Acceptor SMILES']).reshape(i,)
acceptor_homo = np.array(file['HOMO of Acceptor (eV)']).reshape(i,)
acceptor_lumo = np.array(file['LUMO of Acceptor (eV)']).reshape(i,)
acceptor_bandgap = np.array(file['Bandgap of Acceptor (eV)']).reshape(i,)
### Device performance
pce = np.array(file['PCE (%)']).reshape(i,)
voc = np.array(file['Voc (V)']).reshape(i,)
jsc = np.array(file['Jsc (mAcm-2)']).reshape(i,)
ff = np.array(file['FF']).reshape(i,)

In [ ]:
print(f'-----Categories-----\nDevice: {len(np.unique(donor_smiles+acceptor_smiles))}\nDonor materials: {len(np.unique(donor_smiles))}\nAcceptor materials: {len(np.unique(acceptor_smiles))}')

### Morgan Fingerprint

In [ ]:
### important parameters ###
radius = 5
number_of_bits = 8192

In [ ]:
def MF_transfer(smiles):
    bi = {}
    MF = []
    for i, each_smiles in enumerate(smiles):
        try:
            mol = Chem.MolFromSmiles(each_smiles)
            fp=AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=number_of_bits, bitInfo=bi)
            fp_arr = np.array(fp)
            MF.append(fp_arr)
        except:
            print(f'error smiles index: {index[i]}')
    MF = np.array(MF)
    return MF

In [ ]:
donor_MF = MF_transfer(donor_smiles)
print(donor_MF.shape)
acceptor_MF = MF_transfer(acceptor_smiles)
print(acceptor_MF.shape)
MF = np.hstack((donor_MF,acceptor_MF))
print(MF.shape)

In [ ]:
np.save('donor_MF.npy', donor_MF)

In [ ]:
np.save('acceptor_MF.npy', acceptor_MF)

In [ ]:
np.save('MF.npy', MF)

## test

In [ ]:
donor_MF = np.load('donor_MF.npy')
acceptor_MF = np.load('acceptor_MF.npy')
MF = np.load('MF.npy')

In [ ]:
x = MF
y = pce
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0, train_size=0.8)

In [ ]:
forest = RandomForestRegressor()
model = forest.fit(train_x, train_y)

In [ ]:
def model_evaluate(to_pred_y, predictions):
    pred_y = []
    for p in to_pred_y:
        pred_y.append(p)
    pred_r = np.corrcoef(pred_y, predictions)[0,1].round(3)
    pred_mse = mean_squared_error(to_pred_y, predictions)
    pred_rmse = np.sqrt(pred_mse).round(3)
    return pred_r, pred_rmse

In [ ]:
# model predicting
train_predictions = forest.predict(train_x)
test_predictions = forest.predict(test_x)
total_predictions = forest.predict(x)
train_r, train_rmse = model_evaluate(train_y, train_predictions)
test_r, test_rmse = model_evaluate(test_y, test_predictions)
total_r, total_rmse = model_evaluate(y, total_predictions)

In [ ]:
# rmse, r report
dictionary = {' ':['dataset','RMSE','r'],
              'train':[str(train_x.shape[0]),train_rmse,train_r],
              'test':[str(test_x.shape[0]),test_rmse,test_r],
              'total':[str(x.shape[0]),total_rmse,total_r]}
score_report = pd.DataFrame(dictionary)
display(score_report)

In [ ]:
plt.scatter(train_y, train_predictions)

In [ ]:
plt.scatter(test_y, test_predictions)

In [ ]:
forest.get_params()

In [ ]:
param_grid = {'n_estimators': [100, 300, 500, 1000],
             'max_depth': [10, 50, 100],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
            }

rf = RandomForestRegressor()

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(train_x, train_y)

print(grid_search.best_params_)

In [ ]:
grid_search.cv_results_

In [ ]:
# 創建參數網格
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 創建 RandomForestRegressor
rf = RandomForestRegressor()

# 使用 GridSearchCV 進行超參數調優
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X, y)

# 打印最佳參數
print(grid_search.best_params_)

### training

In [ ]:
# decision tree prediction function
def Decision_Tree_Prediction(x, y, plot_name, model_name, smiles):
    # create folder
    direction = f'./models/{model_name}'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # split dataset
    train_x, test_x = train_test_split(x, random_state=0, train_size=0.8)
    train_y, test_y = train_test_split(y, random_state=0, train_size=0.8)
    # model training
    param_grid = {'max_depth':np.arange(75,125,2)}
    model = GridSearchCV(DecisionTreeRegressor(), param_grid, scoring='neg_mean_squared_error', n_jobs=-1,
                         refit=True, cv=5, verbose=1).fit(train_x, train_y)
    train_result = pd.DataFrame(model.cv_results_)
    tree = model.best_estimator_
    print(f'best_score:{model.best_score_}, splits:{model.n_splits_}, refit_time_:{model.refit_time_}')
    print(model.best_params_)
    # model predicting
    train_predictions = tree.predict(train_x)
    test_predictions = tree.predict(test_x)
    total_predictions = tree.predict(x)
    train_r, train_rmse = model_evaluate(train_y, train_predictions)
    test_r, test_rmse = model_evaluate(test_y, test_predictions)
    total_r, total_rmse = model_evaluate(y, total_predictions)
    # rmse, r report
    dictionary = {' ':['dataset','RMSE','r'],
                  'train':[str(train_x.shape[0]),train_rmse,train_r],
                  'test':[str(test_x.shape[0]),test_rmse,test_r],
                  'total':[str(x.shape[0]),total_rmse,total_r]}
    score_report = pd.DataFrame(dictionary)
    display(score_report)
    # plot report
    plot_report(train_y, train_predictions, test_y, test_predictions, plot_name, model_name)
    # save model
    ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
    ax.figure.savefig(f'./models/{model_name}/RMSE_and_r_report.jpg', bbox_inches='tight')
    joblib.dump(model, f'./models/{model_name}/{model_name}.pkl')
    
    # create folder
    direction = f'./models/{model_name}/key_factors'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # set parameters
    importances = tree.feature_importances_
    indices = np.argsort(importances)[::-1]
    new_indices = []
    n = 0
    p = 0
    # drawing key factors
    while n < feature_number:
        p_indice = indices[p]
        new_indices.append(p_indice)
        p += 1
        os.mkdir(f'./models/{model_name}/key_factors/{p_indice}')
        n += 1
        for i, smile in enumerate(smiles):
            mol = Chem.MolFromSmiles(smile)
            bi = {}
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
            try:
                sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice, bi)
                sub_struct.save(f'./models/{model_name}/key_factors/{p_indice}/material_{i}.png', bbox_inches='tight')
                #print(p_indice)
                #display(sub_struct)
                next
            except:
                pass
    
    for p_indice in new_indices:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=1, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    
    # plot importances of key factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices])
    plt.xticks(range(feature_number), new_indices, rotation=90)
    plt.ylabel('Importance', fontdict={'family':'Arial','size':8,'weight':'normal'})
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/importances.png', bbox_inches='tight')
    plt.show()

In [ ]:
# decision tree prediction function
def Decision_Tree_Prediction_Perf(x, y, plot_name, model_name, donor_smiles, acceptor_smiles):
    # create folder
    direction = f'./models/{model_name}'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # split dataset
    train_x, test_x = train_test_split(x, random_state=0, train_size=0.8)
    train_y, test_y = train_test_split(y, random_state=0, train_size=0.8)
    # model training
    param_grid = {'max_depth':np.arange(75,125,2)}
    model = GridSearchCV(DecisionTreeRegressor(), param_grid, scoring='neg_mean_squared_error', n_jobs=-1,
                         refit=True, cv=5, verbose=1).fit(train_x, train_y)
    train_result = pd.DataFrame(model.cv_results_)
    tree = model.best_estimator_
    print(f'best_score:{model.best_score_}, splits:{model.n_splits_}, refit_time_:{model.refit_time_}')
    print(model.best_params_)
    # model predicting
    train_predictions = tree.predict(train_x)
    test_predictions = tree.predict(test_x)
    total_predictions = tree.predict(x)
    train_r, train_rmse = model_evaluate(train_y, train_predictions)
    test_r, test_rmse = model_evaluate(test_y, test_predictions)
    total_r, total_rmse = model_evaluate(y, total_predictions)
    # rmse, r report
    dictionary = {' ':['dataset','RMSE','r'],
                  'train':[str(train_x.shape[0]),train_rmse,train_r],
                  'test':[str(test_x.shape[0]),test_rmse,test_r],
                  'total':[str(x.shape[0]),total_rmse,total_r]}
    score_report = pd.DataFrame(dictionary)
    display(score_report)
    # plot report
    plot_report(train_y, train_predictions, test_y, test_predictions, plot_name, model_name)
    # save model
    ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
    ax.figure.savefig(f'./models/{model_name}/RMSE_and_r_report.jpg', bbox_inches='tight')
    joblib.dump(model, f'./models/{model_name}/{model_name}.pkl')
    
    # create folder
    direction = f'./models/{model_name}/key_factors'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # set parameters
    importances = tree.feature_importances_
    indices = np.argsort(importances)[::-1]
    new_indices = []
    n = 0
    p = 0
    # drawing key factors
    while n < feature_number:
        p_indice = indices[p]
        p += 1
        n += 1
        new_indices.append(p_indice)
        os.mkdir(f'./models/{model_name}/key_factors/{p_indice}')
        if p_indice < number_of_bits:
            for i, smile in enumerate(donor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/{p_indice}/material_{i}.png')
                    #print(p_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
        else:
            p_indice_a = p_indice-number_of_bits
            for i, smile in enumerate(acceptor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice_a, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/{p_indice}/material_{i}.png')
                    #print(p_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
    
    for p_indice in new_indices:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=1, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    
    # plot importances of kay factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices])
    plt.xticks(range(feature_number), new_indices, rotation=90)
    plt.ylabel('Importance', fontdict={'family':'Arial','size':8,'weight':'normal'})
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/importances.png', bbox_inches='tight')
    plt.show()
    
    
    # create folders
    direction_1 = f'./models/{model_name}/key_factors/Donor'
    direction_2 = f'./models/{model_name}/key_factors/Acceptor'
    if not os.path.exists(direction_1):
        os.mkdir(direction_1)
    else:
        shutil.rmtree(direction_1)
        os.mkdir(direction_1)
    if not os.path.exists(direction_2):
        os.mkdir(direction_2)
    else:
        shutil.rmtree(direction_2)
        os.mkdir(direction_2)
    # set parameters
    new_indices = []
    new_indices_a = []
    n = 0
    p = 0
    q = 0
    m = 0
    # drawing key factors of donor
    while n < feature_number:
        p_indice = indices[p]
        p += 1
        if p_indice < number_of_bits:
            n += 1
            os.mkdir(f'./models/{model_name}/key_factors/Donor/{p_indice}')
            new_indices.append(p_indice)
            for i, smile in enumerate(donor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/Donor/{p_indice}/material_{i}.png')
                    #print(p_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
        else:
            pass
    
    for p_indice in new_indices:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=2, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/Donor/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    # plot importances of key factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance for Donor', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices])
    plt.xticks(range(feature_number), new_indices, rotation=90)
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/Donor/importances.png', bbox_inches='tight')
    plt.show()
    
    # drawing hey factors of acceptor
    while m < feature_number:
        p_indice = indices[q]
        q += 1
        if p_indice > number_of_bits:
            m += 1
            new_indices_a.append(p_indice)
            os.mkdir(f'./models/{model_name}/key_factors/Acceptor/{p_indice}')
            p_indice_a = p_indice-number_of_bits
            for i, smile in enumerate(acceptor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice_a, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/Acceptor/{p_indice}/material_{i}.png')
                    #print(x_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
        else:
            pass
    
    for p_indice in new_indices_a:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=3, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/Acceptor/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    
    # plot importances of key factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance for Acceptor', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices_a])
    plt.xticks(range(feature_number), new_indices_a, rotation=90)
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/Acceptor/importances.png', bbox_inches='tight')
    plt.show()

In [ ]:
# random forest function
def Random_Forest_Prediction(x, y, plot_name, model_name, smiles):
    # creste folder
    direction = f'./models/{model_name}'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # split dataset
    train_x, test_x = train_test_split(x, random_state=0, train_size=0.8)
    train_y, test_y = train_test_split(y, random_state=0, train_size=0.8)
    # model training and predicting
    forest = RandomForestRegressor()
    model = forest.fit(train_x, train_y)
    # model predicting
    train_predictions = forest.predict(train_x)
    test_predictions = forest.predict(test_x)
    total_predictions = forest.predict(x)
    train_r, train_rmse = model_evaluate(train_y, train_predictions)
    test_r, test_rmse = model_evaluate(test_y, test_predictions)
    total_r, total_rmse = model_evaluate(y, total_predictions)
    # rmse, r report
    dictionary = {' ':['dataset','RMSE','r'],
                  'train':[str(train_x.shape[0]),train_rmse,train_r],
                  'test':[str(test_x.shape[0]),test_rmse,test_r],
                  'total':[str(x.shape[0]),total_rmse,total_r]}
    score_report = pd.DataFrame(dictionary)
    display(score_report)
    # plot report
    plot_report(train_y, train_predictions, test_y, test_predictions, plot_name, model_name)
    # save model
    ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
    ax.figure.savefig(f'./models/{model_name}/RMSE_and_r_report.jpg', bbox_inches='tight')
    joblib.dump(model, f'./models/{model_name}/{model_name}.pkl')
    
    # create folder
    direction = f'./models/{model_name}/key_factors'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # set parameters
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    new_indices = []
    n = 0
    p = 0
    # drawing key factors
    while n < feature_number:
        p_indice = indices[p]
        p += 1
        n += 1
        new_indices.append(p_indice)
        os.mkdir(f'./models/{model_name}/key_factors/{p_indice}')
        for i, smile in enumerate(smiles):
            mol = Chem.MolFromSmiles(smile)
            bi = {}
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
            try:
                sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice, bi)
                sub_struct.save(f'./models/{model_name}/key_factors/{p_indice}/material_{i}.png', bbox_inches='tight')
                #print(p_indice)
                #display(sub_struct)
                next
            except:
                pass
    
    for p_indice in new_indices:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=1, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    
    # plot importances of key factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices])
    plt.xticks(range(feature_number), new_indices, rotation=90)
    plt.ylabel('Importance', fontdict={'family':'Arial','size':8,'weight':'normal'})
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/importances.png', bbox_inches='tight')
    plt.show()

In [ ]:
# random forest function
def Random_Forest_Prediction_Perf(x, y, plot_name, model_name, donor_smiles, acceptor_smiles):
    # creste folder
    direction = f'./models/{model_name}'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # split dataset
    train_x, test_x = train_test_split(x, random_state=0, train_size=0.8)
    train_y, test_y = train_test_split(y, random_state=0, train_size=0.8)
    # model training and predicting
    forest = RandomForestRegressor()
    model = forest.fit(train_x, train_y)
    # model predicting
    train_predictions = forest.predict(train_x)
    test_predictions = forest.predict(test_x)
    total_predictions = forest.predict(x)
    train_r, train_rmse = model_evaluate(train_y, train_predictions)
    test_r, test_rmse = model_evaluate(test_y, test_predictions)
    total_r, total_rmse = model_evaluate(y, total_predictions)
    # rmse, r report
    dictionary = {' ':['dataset','RMSE','r'],
                  'train':[str(train_x.shape[0]),train_rmse,train_r],
                  'test':[str(test_x.shape[0]),test_rmse,test_r],
                  'total':[str(x.shape[0]),total_rmse,total_r]}
    score_report = pd.DataFrame(dictionary)
    display(score_report)
    # plot report
    plot_report(train_y, train_predictions, test_y, test_predictions, plot_name, model_name)
    # save model
    ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
    ax.figure.savefig(f'./models/{model_name}/RMSE_and_r_report.jpg', bbox_inches='tight')
    joblib.dump(model, f'./models/{model_name}/{model_name}.pkl')
    
    # create folder
    direction = f'./models/{model_name}/key_factors'
    if not os.path.exists(direction):
        os.mkdir(direction)
    else:
        shutil.rmtree(direction)
        os.mkdir(direction)
    # set parameters
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    new_indices = []
    n = 0
    p = 0
    # drawing key factors
    while n < feature_number:
        p_indice = indices[p]
        p += 1
        n += 1
        os.mkdir(f'./models/{model_name}/key_factors/{p_indice}')
        new_indices.append(p_indice)
        if p_indice < number_of_bits:
            for i, smile in enumerate(donor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/{p_indice}/materila_{i}.png')
                    #print(p_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
        else:
            p_indice_a = p_indice-number_of_bits
            for i, smile in enumerate(acceptor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice_a, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/{p_indice}/material_{i}.png')
                    #print(p_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
   
    for p_indice in new_indices:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=1, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    
    # plot importances of kay factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices])
    plt.xticks(range(feature_number), new_indices, rotation=90)
    plt.ylabel('Importance', fontdict={'family':'Arial','size':8,'weight':'normal'})
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/importances.png', bbox_inches='tight')
    plt.show()
    
    
    # create folders
    direction_1 = f'./models/{model_name}/key_factors/Donor'
    direction_2 = f'./models/{model_name}/key_factors/Acceptor'
    if not os.path.exists(direction_1):
        os.mkdir(direction_1)
    else:
        shutil.rmtree(direction_1)
        os.mkdir(direction_1)
    if not os.path.exists(direction_2):
        os.mkdir(direction_2)
    else:
        shutil.rmtree(direction_2)
        os.mkdir(direction_2)
    # set parameters
    new_indices = []
    new_indices_a = []
    n = 0
    p = 0
    q = 0
    m = 0
    # drawing key factors of donor
    while n < feature_number:
        p_indice = indices[p]
        p += 1
        if p_indice < number_of_bits:
            new_indices.append(p_indice)
            n += 1
            os.mkdir(f'./models/{model_name}/key_factors/Donor/{p_indice}')
            for i, smile in enumerate(donor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/Donor/{p_indice}/material_{i}.png')
                    #print(p_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
        else:
            pass
    
    for p_indice in new_indices:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=2, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/Donor/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    # plot importances of key factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance for Donor', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices])
    plt.xticks(range(feature_number), new_indices, rotation=90)
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/Donor/importances.png', bbox_inches='tight')
    plt.show()
    
    # drawing hey factors of acceptor
    while m < feature_number:
        p_indice = indices[q]
        q += 1
        if p_indice > number_of_bits:
            m += 1
            new_indices_a.append(p_indice)
            os.mkdir(f'./models/{model_name}/key_factors/Acceptor/{p_indice}')
            p_indice_a = p_indice-number_of_bits
            for i, smile in enumerate(acceptor_smiles):
                mol = Chem.MolFromSmiles(smile)
                bi = {}
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, number_of_bits, bitInfo=bi)
                try:
                    sub_struct = Chem.Draw.DrawMorganBit(mol, p_indice_a, bi)
                    sub_struct.save(f'./models/{model_name}/key_factors/Acceptor/{p_indice}/material_{i}.png')
                    #print(x_indice)
                    #display(sub_struct)
                    next
                except:
                    pass
        else:
            pass
    
    for p_indice in new_indices_a:
        key_MF_list = []
        key_y_list = []
        na_key_MF_list = []
        na_key_y_list = []
        for each_MF,each_y in zip(x,y):
            if each_MF[p_indice] == 1:
                key_MF_list.append(each_MF)
                key_y_list.append(each_y)
            elif each_MF[p_indice] == 0:
                na_key_MF_list.append(each_MF)
                na_key_y_list.append(each_y)
            else:
                pass
        key_MF = np.array(key_MF_list)
        na_key_MF = np.array(na_key_MF_list)
        key_y = np.array(key_y_list)
        na_key_y = np.array(na_key_y_list)
        key_predictions = model.predict(key_MF)
        na_key_predictions = model.predict(na_key_MF)
        key_r, key_rmse = model_evaluate(key_y, key_predictions)
        na_key_r, na_key_rmse = model_evaluate(na_key_y, na_key_predictions)
        dictionary = {'':['dataset','RMSE','r'],
                      'key factor set':[str(key_MF.shape[0]),key_rmse,key_r],
                      'NA key factor set':[str(na_key_MF.shape[0]),na_key_rmse,na_key_r],
                      'total':[str(x.shape[0]),total_rmse,total_r]}
        score_report = pd.DataFrame(dictionary)
        display(score_report)
        # plot report
        plot_report(na_key_y, na_key_predictions, key_y, key_predictions, plot_name, model_name, plot_mode=3, p_indice=p_indice)
        # save model
        ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
        ax.figure.savefig(f'./models/{model_name}/key_factors/Acceptor/{p_indice}_RMSE_and_r_report.jpg', bbox_inches='tight')
    
    # plot importances of key factors
    plt.figure(figsize=(3,2), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    #plt.title('Feature Importance for Acceptor', fontsize=16)
    plt.bar(range(feature_number), importances[new_indices_a])
    plt.xticks(range(feature_number), new_indices_a, rotation=90)
    ax = plt.gca()
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.5)
    ### ax.ticklabel_format(style='sci')
    ax.tick_params(axis='both', which='major', length=5, width=1.5, labelsize=8, direction='in')
    plt.savefig(f'./models/{model_name}/key_factors/Acceptor/importances.png', bbox_inches='tight')
    plt.show()

In [ ]:
# plot function
def plot_report(train_y, train_predictions, test_y, test_predictions, plot_name, model_name, normalize='no', plot_mode=0, p_indice='na'):
    max_y = max([max(train_y),max(test_y)])
    min_y = min([min(train_y),min(test_y)])
    plot_name_x = plot_name[0]
    plot_name_y = plot_name[1]
    plt.figure(figsize=(3,3), dpi=300)
    plt.rc('font', family='Arial', weight='normal')
    if plot_mode == 0:
        plt.scatter(train_y, train_predictions, 1.4, c='b', marker='o', label='train')
        plt.scatter(test_y, test_predictions, 1.4, c='r', marker='s', label='test')
    else:
        plt.scatter(train_y, train_predictions, 1.4, c='b', marker='o', label='no key factor')
        plt.scatter(test_y, test_predictions, 1.4, c='r', marker='s', label='key factor')
    plt.legend(fontsize=8)
    
    if normalize=='no':
        if max_y > 0 and max_y < 1:
            plot_x = np.arange(0,1.1,0.1)
            plt.xlim(0,1)
            plt.ylim(0,1)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=1)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(5))
            ax.yaxis.set_major_locator(MaxNLocator(5))
            ax.xaxis.set_minor_locator(AutoMinorLocator(2))
            ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        elif max_y > 30 and min_y < 0.5:
            plot_x = np.arange(0,36,1)
            plt.xlim(0,35)
            plt.ylim(0,35)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=1)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(4))
            ax.yaxis.set_major_locator(MaxNLocator(4))
            ax.xaxis.set_minor_locator(AutoMinorLocator(5))
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))

        elif max_y > 15 and min_y < 0.5:
            plot_x = np.arange(0,21,1)
            plt.xlim(0,20)
            plt.ylim(0,20)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=1)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(5))
            ax.yaxis.set_major_locator(MaxNLocator(5))
            ax.xaxis.set_minor_locator(AutoMinorLocator(5))
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))

        elif max_y > 1 and min_y < 0.5:
            plot_x = np.arange(0,1.6,0.1)
            plt.xlim(0,1.5)
            plt.ylim(0,1.5)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=1)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(7))
            ax.yaxis.set_major_locator(MaxNLocator(7))
            ax.xaxis.set_minor_locator(AutoMinorLocator(5))
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))

        elif max_y > 0:
            plot_x = np.arange(0.5,3.6,0.1)
            plt.xlim(0.5,3.5)
            plt.ylim(0.5,3.5)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=1)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(7))
            ax.yaxis.set_major_locator(MaxNLocator(7))
            ax.xaxis.set_minor_locator(AutoMinorLocator(5))
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))

        elif max_y < -3.5:
            plot_x = np.arange(-7,-2,1)
            plt.xlim(-7,-3)
            plt.ylim(-7,-3)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=0.8)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(5))
            ax.yaxis.set_major_locator(MaxNLocator(5))
            ax.xaxis.set_minor_locator(AutoMinorLocator(5))
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))

        else:
            plot_x = np.arange(-5,-1,1)
            plt.xlim(-5,-2)
            plt.ylim(-5,-2)
            plot_y = plot_x
            plt.plot(plot_x, plot_y, '--k', linewidth=1)
            ax = plt.gca()
            ax.set_aspect('equal', adjustable='box', anchor='C')
            ax.xaxis.set_major_locator(MaxNLocator(4))
            ax.yaxis.set_major_locator(MaxNLocator(4))
            ax.xaxis.set_minor_locator(AutoMinorLocator(5))
            ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        for axis in ['top', 'bottom', 'left', 'right']:
            ax.spines[axis].set_linewidth(1.5)
        ax.tick_params(axis='both', which='major', length=4.5, width=1.5, labelsize=8, direction='in')
        ax.tick_params(axis='both', which='minor', length=3.5, width=1, direction='in')
        plt.xlabel(plot_name_x, fontdict={'family':'Arial','size':8,'weight':'normal'})
        plt.ylabel(plot_name_y, fontdict={'family':'Arial','size':8,'weight':'normal'})
        if plot_mode == 0:
            plt.savefig(f'./models/{model_name}/regression_plot_report.jpg', bbox_inches='tight')
            plt.show()
        elif plot_mode == 1:
            plt.savefig(f'./models/{model_name}/key_factors/{p_indice}_regression_plot_report.jpg', bbox_inches='tight')
            plt.show()
        elif plot_mode == 2:
            plt.savefig(f'./models/{model_name}/key_factors/Donor/{p_indice}_regression_plot_report.jpg', bbox_inches='tight')
            plt.show()
        elif plot_mode == 3:
            plt.savefig(f'./models/{model_name}/key_factors/Acceptor/{p_indice}_regression_plot_report.jpg', bbox_inches='tight')
            plt.show()
        else:
            pass
    else:
        plot_x = np.arange(0,1.1,0.1)
        plt.xlim(0,1)
        plt.ylim(0,1)
        plot_y = plot_x
        plt.plot(plot_x, plot_y, '--k', linewidth=1)
        ax = plt.gca()
        ax.set_aspect('equal', adjustable='box', anchor='C')
        ax.xaxis.set_major_locator(MaxNLocator(5))
        ax.yaxis.set_major_locator(MaxNLocator(5))
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        for axis in ['top', 'bottom', 'left', 'right']:
            ax.spines[axis].set_linewidth(1.5)
        ax.tick_params(axis='both', which='major', length=4.5, width=1.5, labelsize=8, direction='in')
        ax.tick_params(axis='both', which='minor', length=3.5, width=1, direction='in')
        plt.xlabel(plot_name_x, fontdict={'family':'Arial','size':8,'weight':'normal'})
        plt.ylabel(plot_name_x, fontdict={'family':'Arial','size':8,'weight':'normal'})
        if plot_mode == 0:
            plt.savefig(f'./models/{model_name}/regression_plot_report_denormalized.jpg', bbox_inches='tight')
            plt.show()
        elif plot_mode == 1:
            plt.savefig(f'./models/{model_name}/key_factors/{p_indice}_regression_plot_report_denormalized.jpg', bbox_inches='tight')
            plt.show()
        elif plot_mode == 2:
            plt.savefig(f'./models/{model_name}/key_factors/Donor/{p_indice}_regression_plot_report_denormalized.jpg', bbox_inches='tight')
            plt.show()
        elif plot_mode == 3:
            plt.savefig(f'./models/{model_name}/key_factors/Acceptor/{p_indice}_regression_plot_report_denormalized.jpg', bbox_inches='tight')
            plt.show()
        else:
            pass

In [ ]:
def draw_save_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='k',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)
    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
    return ax

In [ ]:
# r and rmse
def model_evaluate(to_pred_y, predictions):
    pred_y = []
    for p in to_pred_y:
        pred_y.append(p)
    pred_r = np.corrcoef(pred_y, predictions)[0,1].round(3)
    pred_mse = mean_squared_error(to_pred_y, predictions)
    pred_rmse = np.sqrt(pred_mse).round(3)
    return pred_r, pred_rmse

In [ ]:
feature_number = 20

In [ ]:
model_name = 'RF_MF_PCE'
plot_name = ['Experimental PCE (%)','Predicted PCE (%)']
Random_Forest_Prediction_Perf(MF, pce, plot_name, model_name, donor_smiles, acceptor_smiles)

In [ ]:
x = MF
y = pce
# split dataset
train_x, test_x = train_test_split(x, random_state=0, train_size=0.8)
train_y, test_y = train_test_split(y, random_state=0, train_size=0.8)


In [ ]:
# model training and predicting
forest = RandomForestRegressor()
model = forest.fit(train_x, train_y)
# model predicting
train_predictions = forest.predict(train_x)
test_predictions = forest.predict(test_x)
total_predictions = forest.predict(x)
train_r, train_rmse = model_evaluate(train_y, train_predictions)
test_r, test_rmse = model_evaluate(test_y, test_predictions)
total_r, total_rmse = model_evaluate(y, total_predictions)
# rmse, r report
dictionary = {' ':['dataset','RMSE','r'],
              'train':[str(train_x.shape[0]),train_rmse,train_r],
              'test':[str(test_x.shape[0]),test_rmse,test_r],
              'total':[str(x.shape[0]),total_rmse,total_r]}
score_report = pd.DataFrame(dictionary)
display(score_report)
# plot report
plot_report(train_y, train_predictions, test_y, test_predictions, plot_name, model_name)
# save model
ax = draw_save_table(score_report, header_columns=0, col_width=2.0)
ax.figure.savefig(f'./models/{model_name}/RMSE_and_r_report.jpg', bbox_inches='tight')
joblib.dump(model, f'./models/{model_name}/{model_name}.pkl')

### Random Forest

In [ ]:
model_name = 'RF_MF_Donor_HOMO'
plot_name = ['Experimental HOMO of Donor (eV)','Predicted HOMO of Donor (eV)']
Random_Forest_Prediction(donor_MF, donor_homo, plot_name, model_name, donor_smiles)

In [ ]:
model_name = 'RF_MF_Donor_LUMO'
plot_name = ['Experimental LUMO of Donor (eV)','Predicted LUMO of Donor (eV)']
Random_Forest_Prediction(donor_MF, donor_lumo, plot_name, model_name, donor_smiles)

In [ ]:
model_name = 'RF_MF_Donor_Bandgap'
plot_name = ['Experimental Bandgap of Donor (eV)','Predicted Bandgap of Donor (eV)']
Random_Forest_Prediction(donor_MF, donor_bandgap, plot_name, model_name, donor_smiles)

In [ ]:
model_name = 'RF_MF_Acceptor_HOMO'
plot_name = ['Experimental HOMO of Acceptor (eV)','Predicted HOMO of Acceptor (eV)']
Random_Forest_Prediction(acceptor_MF, acceptor_homo, plot_name, model_name, acceptor_smiles)

In [ ]:
model_name = 'RF_MF_Acceptor_LUMO'
plot_name = ['Experimental LUMO of Acceptor (eV)','Predicted LUMO of Acceptor (eV)']
Random_Forest_Prediction(acceptor_MF, acceptor_lumo, plot_name, model_name, acceptor_smiles)

In [ ]:
model_name = 'RF_MF_Acceptor_Bandgap'
plot_name = ['Experimental Bandgap of Acceptor (eV)','Predicted Bandgap of Acceptor (eV)']
Random_Forest_Prediction(acceptor_MF, acceptor_bandgap, plot_name, model_name, acceptor_smiles)

In [ ]:
model_name = 'RF_MF_PCE'
plot_name = ['Experimental PCE (%)','Predicted PCE (%)']
Random_Forest_Prediction_Perf(MF, pce, plot_name, model_name, donor_smiles, acceptor_smiles)

In [ ]:
model_name = 'RF_MF_Voc'
plot_name = ['Experimental V$_{OC}$ (V)','Predicted V$_{OC}$ (V)']
Random_Forest_Prediction_Perf(MF, voc, plot_name, model_name, donor_smiles, acceptor_smiles)

In [ ]:
model_name = 'RF_MF_Jsc'
plot_name = ['Experimental J$_{SC}$ (mAcm$^{-2}$)','Predicted J$_{SC}$ (mAcm$^{-2}$)']
Random_Forest_Prediction_Perf(MF, jsc, plot_name, model_name, donor_smiles, acceptor_smiles)

In [ ]:
model_name = 'RF_MF_FF'
plot_name = ['Experimental FF', 'Predicted FF']
Random_Forest_Prediction_Perf(MF, ff, plot_name, model_name, donor_smiles, acceptor_smiles)